In [ ]:
# imports
import os
from types import SimpleNamespace
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib.colors as mcolors
import numpy as np

In [ ]:
t2p_save_path = "/Users/manonmantez/Desktop/jm038/t2p_affine_chan1_all_"  # (change this based on your data)
plane = 'plane0' # which plane to process (the example dataset is single-plane)

In [ ]:
# np.load() the match matrix (plane0_match_mat.npy)
t2p_match_mat = np.load(os.path.join(t2p_save_path,'track2p', f'{plane}_match_mat.npy'), allow_pickle=True)

# np.load() settings (this contains suite2p paths etc.) (track_ops.npy)
track_ops_dict = np.load(os.path.join(t2p_save_path, 'track2p', 'track_ops.npy'), allow_pickle=True).item()
track_ops = SimpleNamespace(**track_ops_dict) # create dummy object from the track_ops dictionary

print(t2p_save_path)
print(track_ops.save_path)
print(track_ops.reg_chan)
print(track_ops.transform_type)
print(t2p_match_mat.shape)
#print(t2p_match_mat)

In [ ]:
t2p_match_mat_notnone = t2p_match_mat != None #bolean matrix
print(t2p_match_mat_notnone)
plt.figure(figsize=(20, 1), dpi=500)
plt.imshow(t2p_match_mat_notnone.T, aspect='auto', vmin=0, vmax=1)


In [ ]:
print('Datasets used for t2p:\n')
for ds_path in track_ops.all_ds_path:
    iscell=np.load(os.path.join(ds_path,'suite2p', 'plane0','iscell.npy'))
    iscell = iscell[iscell[:, 0] > track_ops.iscell_thr]
    print(len(iscell))


## Above is the output of the algorithm, now generate ground truth

### Generate grid

In [ ]:
# load the suite2p output of the first dataset
s2p_path = os.path.join(track_ops.all_ds_path[0], 'suite2p', plane)
print(track_ops.all_ds_path)

# load the ops and stat
ops_first = np.load(os.path.join(s2p_path, 'ops.npy'), allow_pickle=True).item()
stat_first = np.load(os.path.join(s2p_path, 'stat.npy'), allow_pickle=True)
iscell_first = np.load(os.path.join(s2p_path, 'iscell.npy'), allow_pickle=True)
s2p_inds = np.arange(len(stat_first))

# filter stat based on the track2p probability
print(f'Filtering stat based on the track2p probability (iscell_thr={track_ops.iscell_thr})')
s2p_inds_iscell = s2p_inds[iscell_first[:, 0] > track_ops.iscell_thr]
stat_first_iscell = stat_first[iscell_first[:, 0] > track_ops.iscell_thr]
print(f'Filtered stat from {len(stat_first)} to {len(stat_first_iscell)} cells')



In [ ]:
# now plot the fov with contours
plt.figure(figsize=(10, 10))
plt.imshow(ops_first['meanImg'], cmap='gray')
for i in range(len(stat_first_iscell)):
    # get random color 
    color = np.random.rand(3)
    plt.scatter(stat_first_iscell[i]['xpix'], stat_first_iscell[i]['ypix'], s=1, color=color, alpha=0.5)

In [ ]:
# now make a 8x8 grid on the image
n_grid = 9
grid_x = np.arange(0, ops_first['meanImg'].shape[1], ops_first['meanImg'].shape[1] // n_grid)
grid_y = np.arange(0, ops_first['meanImg'].shape[0], ops_first['meanImg'].shape[0] // n_grid)

plt.imshow(ops_first['meanImg'], cmap='gray')
for i in range(n_grid):
    plt.axvline(grid_x[i], color='r')
    plt.axhline(grid_y[i], color='r')

plt.axis('off')

In [ ]:
# now for each grid cross get the nearest cell (except for 0 and 512 pixels)
all_cell_med = []

# get all cell medians from stat
for i in range(len(stat_first_iscell)):
    all_cell_med.append(stat_first_iscell[i]['med'])
    

In [ ]:
# compute coordinates of the grid crosses
grid_crosses = []
for i in range(1, n_grid):
    for j in range(1, n_grid):
        grid_crosses.append((grid_x[i], grid_y[j]))

In [ ]:
grid_crosses

In [ ]:
plt.imshow(ops_first['meanImg'], cmap='gray')
for i in range(n_grid):
    plt.axvline(grid_x[i], color='gray')
    plt.axhline(grid_y[i], color='gray')
for cross in grid_crosses:
    plt.scatter(cross[0], cross[1], s=20, color='r')

plt.axis('off')


In [ ]:
# compute distances between each cell and each grid cross
distances = np.zeros((len(stat_first_iscell), len(grid_crosses)))
for i, cell_med in enumerate(all_cell_med):
    for j, cross in enumerate(grid_crosses):
        distances[i, j] = np.sqrt((cell_med[0] - cross[0])**2 + (cell_med[1] - cross[1])**2)

In [ ]:
# now get the nearest cell for each grid cross
nearest_cells = np.argmin(distances, axis=0)

In [ ]:
nearest_cells

In [ ]:
plt.imshow(ops_first['meanImg'], cmap='gray')

for cross in grid_crosses:
    plt.scatter(cross[0], cross[1], s=20, color='C0')

plt.axis('off')

for i, cell in enumerate(nearest_cells):
    roi = np.zeros_like(ops_first['meanImg'])
    roi[stat_first_iscell[cell]['ypix'], stat_first_iscell[cell]['xpix']] = 1
    plt.contour(roi, levels=[0.5], colors='C1')

In [ ]:
# now get the original indices
original_indices = s2p_inds_iscell[nearest_cells]
print('Now open suite2p and try to manually track cells:')
[print(f'{original_indices[i]}') for i in range(len(original_indices))]

In [ ]:
# make the same plot as above but with numbers labeled next to the roi
plt.figure(figsize=(10, 10))
# clip based on 99.99 percentile
img = ops_first['meanImg'].copy()
img[img > np.percentile(img, 99.99)] = np.percentile(img, 99.99)
plt.imshow(img, cmap='gray')

for i, cell in enumerate(nearest_cells):
    roi = np.zeros_like(ops_first['meanImg'])
    roi[stat_first_iscell[cell]['ypix'], stat_first_iscell[cell]['xpix']] = 1
    plt.contour(roi, levels=[0.5], colors='C1')
    plt.text(stat_first_iscell[cell]['med'][1]+10, stat_first_iscell[cell]['med'][0]-10, f'{original_indices[i]}', color='C1')
    plt.scatter(grid_crosses[i][0], grid_crosses[i][1], s=20, color='C0', marker='x')

plt.axis('off')

In [ ]:
np.sort(original_indices)

### Getting real suite2p indexes to compare it to

In [ ]:
def function(path, plane):

    track_ops_dict = np.load(os.path.join(path, "track2p", "track_ops.npy"), allow_pickle=True).item()
    track_ops = SimpleNamespace(**track_ops_dict)
    t2p_match_mat = np.load(os.path.join(path,"track2p" ,f"{plane}_match_mat.npy"), allow_pickle=True)
    #t2p_match_mat_allday = t2p_match_mat
    
    if track_ops.iscell_thr is not None:
        true_indices= np.empty((len(t2p_match_mat), len(track_ops.all_ds_path)), dtype=object)
        
    for j in range(len(t2p_match_mat)):
        for (i, ds_path) in enumerate(track_ops.all_ds_path):
            iscell = np.load(os.path.join(ds_path, 'suite2p', plane, 'iscell.npy'), allow_pickle=True)
            if track_ops.iscell_thr is not None:
                s2p_indexes= np.where(iscell[:,1]>track_ops.iscell_thr)[0]
                t2p_index=t2p_match_mat[j,i]

                if t2p_index is None:
                    true_index = None
                else:
                    true_index=s2p_indexes[t2p_index]

                true_indices[j, i] = true_index
            else:
                pass
    if track_ops.iscell_thr is not None:        
        np.save(os.path.join(path, "true_indices.npy"), true_indices)
        print("saved")
    else:
        print("not in locals")
        

In [ ]:
#only if nerver done for this condition and this mouse before !!!! 
function(t2p_save_path, plane)  

In [ ]:
indices_npy= np.load(os.path.join(t2p_save_path,"true_indices.npy"), allow_pickle=True)
print(indices_npy)
print(indices_npy.shape)
print(t2p_save_path)

#indices_npy (index of the cell in the suite2p output)

In [ ]:
# Configurer numpy pour afficher tout le tableau sans troncature
np.set_printoptions(threshold=np.inf)
print(indices_npy)

In [ ]:
# index rows for cells where the first row is in original_indices
all_s2p_idxs = []
for idx in np.sort(original_indices):
    idx_row = np.where(indices_npy[:,0]==idx)

    if len(idx_row[0]) == 0:
        s2p_idxs = [None]*len(track_ops.all_ds_path)
        #print('here')

    else:
        s2p_idxs = indices_npy[idx_row]
    all_s2p_idxs.append(s2p_idxs[0])

all_s2p_idxs = np.array(all_s2p_idxs)

#print(all_s2p_idxs.dtype)
print(f'all_s2p_idxs {all_s2p_idxs}')

### Loading and processing cellreg outputs

In [ ]:
# check if you are using track2p with channel 1 for each mouse (if not, change the path)
from scipy.io import savemat
import sys

for i, ds_path in enumerate(track_ops.all_ds_path):

    iscell = np.load(os.path.join(ds_path, 'suite2p', f'plane{plane}', 'iscell.npy'), allow_pickle=True)
    stat = np.load(os.path.join(ds_path, 'suite2p', f'plane{plane}', 'stat.npy'), allow_pickle=True)
    ops = np.load(os.path.join(ds_path, 'suite2p', f'plane{plane}', 'ops.npy'), allow_pickle=True).item()
    

    ind_iscell = np.where(iscell[:, 1] > 0.5)[0]  # Extraire les indices dans un tableau
    print(f"Indices des cellules : {ind_iscell}")


    Ly, Lx = int(ops['Ly']), int(ops['Lx'])
    N = len(ind_iscell)  
    print(f"Nombre de cellules : {N}")

    footprints = np.zeros((N, Ly, Lx), dtype=np.int16)

    for neuron_idx, cell_idx in enumerate(ind_iscell):
        cell_stat = stat[cell_idx]

        ypix = cell_stat['ypix']
        xpix = cell_stat['xpix']
        lam = cell_stat['lam']

        footprints[neuron_idx, ypix, xpix] = lam.astype(np.int16)


    save_path_part = os.path.join(ds_path, 'footprints_part.mat')

    print(sys.getsizeof(footprints))
    
    
    savemat(save_path_part, {'footprints_part1': footprints})


In [ ]:
new_path = os.path.dirname(t2p_save_path)
cellreg_path = os.path.join(new_path, 'cellreg')
print(cellreg_path)

In [ ]:
import h5py

# this is the directory that contains a /track2p folder that is output by running the track2p algorithm
# load the data

with h5py.File(os.path.join(cellreg_path, "cellRegistered.mat"), 'r') as f:
    # Accéder au dataset spécifique
    cell_to_index_map = f['cell_registered_struct/cell_to_index_map']
 
    cell_to_index_map = np.array(cell_to_index_map)
    print(cell_to_index_map.shape)

    # Filtrer les colonnes où la première ligne n'est pas égale à 0
    # Pour avoir la même shape que le tableau de match de track2p (autant de lignes que dans iscell > 0.5 du jour 1)
    mask0 = cell_to_index_map[0, :] != 0
    filtered_cell_to_index_map = cell_to_index_map[:, mask0]
    print(filtered_cell_to_index_map.astype(int))

    # Convertir en entier
    filtered_cell_to_index_map = filtered_cell_to_index_map.astype(int)

    # Remplacer les zéros par None et décrémenter les autres chiffres de 1
    filtered_cell_to_index_map = np.where(filtered_cell_to_index_map == 0, None, filtered_cell_to_index_map - 1)
    print(filtered_cell_to_index_map)

    # Enregistrer le tableau modifié
    np.save(os.path.join(cellreg_path, "plane0_match_mat_cellreg.npy"), filtered_cell_to_index_map)
    print("NumPy file 'plane0_match_mat_cellreg.npy' created successfully.")


In [ ]:
cellreg_match_mat = np.load(os.path.join(cellreg_path, "plane0_match_mat_cellreg.npy"), allow_pickle=True)
print(cellreg_match_mat)
print(cellreg_match_mat.shape)

In [ ]:
# Afficher le tableau modifié avec plt.imshow
bool_array = np.where(cellreg_match_mat == None, 0, 1).astype(bool)
plt.figure(figsize=(20, 1), dpi=500)
plt.imshow(bool_array,  aspect='auto', vmin=0, vmax=1)
plt.show()

In [ ]:
cellreg_match_mat = cellreg_match_mat.T

In [ ]:
cellreg_match_mat

In [ ]:
for i in range(len(cellreg_match_mat)):
    if all(x is None for x in cellreg_match_mat[i][1:]):
        cellreg_match_mat[i] = None

In [ ]:
cellreg_match_mat

In [ ]:
# For same as t2p_match_mat
for i in range(len(cellreg_match_mat)):
    if cellreg_match_mat[i,1] is None:
        cellreg_match_mat[i] = None

#ne pas faire si cellreg pairs


In [ ]:
cellreg_match_mat

#### Getting real suite2p indexes to compare it to

In [ ]:
def function_cellreg(path, plane):

    if track_ops.iscell_thr is not None:
        true_indices= np.empty(cellreg_match_mat.shape, dtype=object)
        print(true_indices.shape)
        
    for (i, ds_path) in enumerate(track_ops.all_ds_path):
        iscell = np.load(os.path.join(ds_path, 'suite2p', plane, 'iscell.npy'), allow_pickle=True)
        s2p_indexes= np.where(iscell[:,1]>track_ops.iscell_thr)[0]
        #print(s2p_indexes)
        print(f'suite2P {len(s2p_indexes)}')
        for j in range(cellreg_match_mat.shape[0]):
                if track_ops.iscell_thr is not None:
                    cellreg_index=cellreg_match_mat[j,i]
                    print(cellreg_index)

                if cellreg_index is None :
                    true_index = None
                else:
                    true_index=s2p_indexes[cellreg_index]

                true_indices[j, i] = true_index
        else:
            pass
    if track_ops.iscell_thr is not None:        
        np.save(os.path.join(path, "true_indices_cellreg.npy"), true_indices)
        print("saved")
    else:
        print("not in locals")
        

In [ ]:
function_cellreg(cellreg_path, plane)  

In [ ]:
indices_cellreg_npy= np.load(os.path.join(cellreg_path,"true_indices_cellreg.npy"), allow_pickle=True)
print(indices_cellreg_npy)
print(indices_cellreg_npy.shape)

In [ ]:
# index rows for cells where the first row is in original_indices
all_s2p_idxs_cellreg = []
for idx in np.sort(original_indices):
    idx_row = np.where(indices_cellreg_npy[:,0]==idx)
    print(idx_row)

    if len(idx_row[0]) == 0:
        s2p_idxs = [None]*len(track_ops.all_ds_path)
        #print('here')

    else:
        s2p_idxs = indices_cellreg_npy[idx_row]
    all_s2p_idxs_cellreg.append(s2p_idxs[0])

all_s2p_idxs_cellreg = np.array(all_s2p_idxs_cellreg)


In [ ]:
all_s2p_idxs_cellreg

### Loading and processing 'ground truth' 

In [ ]:
# load the data
new_path = os.path.dirname(t2p_save_path)
print(new_path)

table = np.genfromtxt(os.path.join(new_path, 'ground_truth.csv'), delimiter=';', skip_header=1)
sorted_indices = np.argsort(table[:, 0])
table = table[sorted_indices]
table = np.where(np.isnan(table), None, table)
table = [[int(x) if x is not None else None for x in row] for row in table]
table

In [ ]:
# same as track2p
for i in range(len(table)):
    if all(x is None for x in table[i][1:]):
        table[i] = None

print(table)

In [ ]:
for i in range(len(table)):
    # print the rows
    print(f'track2p:     {all_s2p_idxs[i]}')
    print(f'manual:      {table[i]}')
    print('')

## Processing track2p and ground truth and compute metrics

In [ ]:
def compute_performance_criteria(all_s2p_idxs_without_none, table_without_none):

    #Tgt : number of all references tracks 
    #Tc : number of all computed tracks
    #Trc : number of completely reconstructed reference tracks 

    Tgt= len(table_without_none)
    print(f"Tgt : {Tgt}")
    Tc= len(all_s2p_idxs_without_none)
    print(f"Tc : {Tc}")
    Trc= 0

    for line in table_without_none:
        found = any(all(elem in row for elem in line) for row in all_s2p_idxs_without_none if row is not None)
        if found:
            #print(f"Tous les éléments de la ligne {line} sont retrouvés dans all_s2p_idxs_without_none.")
            Trc += 1
        else:
            print(f"Tous les éléments de la ligne {line} ne sont pas retrouvés dans all_s2p_idxs_without_none.")

    CT= (2*Trc)/(Tgt+Tc)
    Accuracy= Trc/Tgt
    print(f"Trc : {Trc}")
    print(f"CT : { round(CT, 2)}")
    print(f"Accuracy : {round(Accuracy,2)}")
    
    
    return CT, Accuracy


### Only cells present on X days of analysis separately for GT and Track2p

In [ ]:
CT_values= []
accuracy_values= []

for i in range (2, t2p_match_mat.shape[1] + 1):
    all_s2p_idxs_copy = all_s2p_idxs.copy()
    table_copy=table.copy()
    
    filtered_all_s2p_idxs = [row[:i] if row is not None else None for row in all_s2p_idxs_copy]
    filtered_table_copy = [row[:i] if row is not None else None for row in table_copy]

    for i in range(len(filtered_all_s2p_idxs)):
        if filtered_all_s2p_idxs[i] is not None: 
            if any(x is None for x in filtered_all_s2p_idxs[i]):
                    filtered_all_s2p_idxs[i] = None

    for i in range(len(filtered_table_copy)):
        if filtered_table_copy[i] is not None:
                if any(x is None for x in filtered_table_copy[i]):
                    filtered_table_copy[i] = None

    count = sum(1 for row in filtered_table_copy if row is not None)
    print(f'GT {count}')
    count = sum(1 for row in filtered_all_s2p_idxs if row is not None)
    print(f'Track2p {count}')

    table_without_none = [x for x in filtered_table_copy if x is not None]
    all_s2p_idxs_without_none = [x for x in filtered_all_s2p_idxs if x is not None]
    print(len(table_without_none))
    print(len(all_s2p_idxs_without_none))

    Ct, accuracy = compute_performance_criteria (all_s2p_idxs_without_none, table_without_none)
    CT_values.append(round(Ct,2))
    accuracy_values.append(round(accuracy,2))

    print('')

result = np.empty((2, len(CT_values) + 1), dtype=object)
result[0, 0] = "CT"
result[1, 0] = "accuracy"
result[0, 1:] = CT_values
result[1, 1:] = accuracy_values
print(result)

np.save(os.path.join(t2p_save_path, 'result_CT.npy'), result)

### Only the cells presents on all days in the GT (table) and evaluation from last day to D0 (first day of analysis) 

In [ ]:
table_cpy=table.copy()
for i in range(len(table_cpy)):
        if table_cpy[i] is not None: 
            if any(x is None for x in table_cpy[i]):
                    table_cpy[i] = None
table_all_days = [x for x in table_cpy if x is not None]
indices_all_days = [i for i, x in enumerate(table_cpy) if x is not None]
print(len(table_all_days))

all_s2p_idxs_cpy=all_s2p_idxs.copy()
all_s2p_idxs_gt = all_s2p_idxs_cpy[indices_all_days]
print(len(all_s2p_idxs_gt))

CT_values= []
accuracy_values= []

for i in range (2, t2p_match_mat.shape[1] + 1):
        all_s2p_idxs_i = [row[:i] if row is not None else None for row in all_s2p_idxs_gt]
        print(all_s2p_idxs_i)
        table_i= [row[:i] if row is not None else None for row in table_all_days]
        print(table_i)
        Ct, accuracy= compute_performance_criteria(all_s2p_idxs_i, table_i)
        CT_values.append(round(Ct,2))
        accuracy_values.append(round(Ct,2))

result = np.empty((2, len(CT_values) + 1), dtype=object)
result[0, 0] = "CT"
result[1, 0] = "accuracy"
result[0, 1:] = CT_values
result[1, 1:] = accuracy_values
print(result)

np.save(os.path.join(t2p_save_path, 'result_CT_GT.npy'), result)


In [ ]:
print(np.load(os.path.join(t2p_save_path, 'result_CT.npy'), allow_pickle=True))

In [ ]:
print(np.load(os.path.join(t2p_save_path, 'result_CT_GT.npy'), allow_pickle=True))

In [ ]:
original_indices_sorted=np.sort(original_indices)
original_indices_all_days = original_indices_sorted[indices_all_days]
print(original_indices_all_days)
nearest_cells_sorted=np.sort(nearest_cells)
nearest_cells_all_days = nearest_cells_sorted[indices_all_days]
print(nearest_cells_all_days)
# make the same plot as above but with numbers labeled next to the roi
plt.figure(figsize=(10, 10))
# clip based on 99.99 percentile
img = ops_first['meanImg'].copy()
img[img > np.percentile(img, 99.99)] = np.percentile(img, 99.99)
plt.imshow(img, cmap='gray')

for i, cell in enumerate(nearest_cells_all_days):
    roi = np.zeros_like(ops_first['meanImg'])
    roi[stat_first_iscell[cell]['ypix'], stat_first_iscell[cell]['xpix']] = 1
    plt.contour(roi, levels=[0.5], colors='C1')
    plt.text(stat_first_iscell[cell]['med'][1]+10, stat_first_iscell[cell]['med'][0]-10, f'{original_indices_all_days[i]}', color='C1')
    #plt.scatter(grid_crosses[i][0], grid_crosses[i][1], s=20, color='C0', marker='x')

plt.axis('off')


### As above, but for cellreg

In [ ]:
CT_values= []
accuracy_values= []

for i in range (2, t2p_match_mat.shape[1] + 1):
    all_s2p_idxs_cellreg_copy = all_s2p_idxs_cellreg.copy()
    table_copy=table.copy()
    
    filtered_all_s2p_idxs_cellreg = [row[:i] if row is not None else None for row in all_s2p_idxs_cellreg_copy]
    filtered_table_copy = [row[:i] if row is not None else None for row in table_copy]

    for i in range(len(filtered_all_s2p_idxs_cellreg)):
        if filtered_all_s2p_idxs_cellreg[i] is not None: 
            if any(x is None for x in filtered_all_s2p_idxs_cellreg[i]):
                    filtered_all_s2p_idxs_cellreg[i] = None

    for i in range(len(filtered_table_copy)):
        if filtered_table_copy[i] is not None:
                if any(x is None for x in filtered_table_copy[i]):
                    filtered_table_copy[i] = None

    count = sum(1 for row in filtered_table_copy if row is not None)
    print(f'GT {count}')
    count = sum(1 for row in filtered_all_s2p_idxs_cellreg if row is not None)
    print(f'Track2p {count}')

    table_without_none = [x for x in filtered_table_copy if x is not None]
    all_s2p_idxs_cellreg_none = [x for x in filtered_all_s2p_idxs_cellreg if x is not None]
    print(len(table_without_none))
    print(len(all_s2p_idxs_cellreg_none))

    Ct, accuracy = compute_performance_criteria (all_s2p_idxs_cellreg_none, table_without_none)
    CT_values.append(round(Ct,2))
    accuracy_values.append(round(accuracy,2))

    print('')

result = np.empty((2, len(CT_values) + 1), dtype=object)
result[0, 0] = "CT"
result[1, 0] = "accuracy"
result[0, 1:] = CT_values
result[1, 1:] = accuracy_values
print(result)

np.save(os.path.join(cellreg_path, 'result_CT.npy'), result)
    

In [ ]:
table_cpy=table.copy()
for i in range(len(table_cpy)):
        if table_cpy[i] is not None: 
            if any(x is None for x in table_cpy[i]):
                    table_cpy[i] = None
table_all_days = [x for x in table_cpy if x is not None]
indices_all_days = [i for i, x in enumerate(table_cpy) if x is not None]
print(len(table_all_days))

all_s2p_idxs_cellreg_copy=all_s2p_idxs_cellreg.copy()
all_s2p_idxs_cellreg_gt = all_s2p_idxs_cellreg_copy[indices_all_days]
print(len(all_s2p_idxs_cellreg_gt))

CT_values= []
accuracy_values= []

for i in range (2, t2p_match_mat.shape[1] + 1):
        all_s2p_idxs_i = [row[:i] if row is not None else None for row in all_s2p_idxs_cellreg_gt]
        table_i= [row[:i] if row is not None else None for row in table_all_days]
        Ct, accuracy= compute_performance_criteria(all_s2p_idxs_i, table_i)
        CT_values.append(round(Ct,2))
        accuracy_values.append(round(Ct,2))

result = np.empty((2, len(CT_values) + 1), dtype=object)
result[0, 0] = "CT"
result[1, 0] = "accuracy"
result[0, 1:] = CT_values
result[1, 1:] = accuracy_values
print(result)

np.save(os.path.join(cellreg_path, 'result_CT_GT.npy'), result)